# <center> TSA Account Closure First Notification</center>

In [1]:
# Import Libraries   

import pandas as pd  
import numpy as np  

from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

In [2]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID 35388d50-45df-49ec-8440-9b520707eac0 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2024-03-24,536736


In [4]:
from datetime import datetime, timedelta

# Given date
given_date = datetime.strptime('2024-03-25', '%Y-%m-%d')

# Calculate date 60 days ago
result_date = given_date - timedelta(days=60)

# Format and print the result
formatted_result_date = result_date.strftime('%Y-%m-%d')
print(f"The d-60 days is {formatted_result_date}")


The d-60 days is 2024-01-25


In [5]:
from datetime import datetime, timedelta

# Given date
given_date = datetime.strptime('2024-03-25', '%Y-%m-%d')

# Calculate date 180 days ago
result_date = given_date - timedelta(days=180)

# Format and print the result
formatted_result_date = result_date.strftime('%Y-%m-%d')
print(f"The d-180 days is {formatted_result_date}")

The d-180 days is 2023-09-27


The date should be D-1? If the date is not D-1 then ask Bala to refresh the table.

Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

First Notification we change all the steps <br><br>
Second and Third Notification we only change Step1 and Step5
For Second Notificaiton and Third Notificaiton the customre id should be from the file sent by Marvin. <br><br>

In [7]:
sq = """with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
)
,
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2024-03-24'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
)
,
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select 
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '2024-01-25')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'


    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2024-01-25')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and TSA_Opening_date < date '2023-09-27'  and (Last_TSA_Credit_Date < date '2023-09-27' and Last_TSA_Debit_Date < date '2023-09-27' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') 
    < date '2023-09-27')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and tsa_status = 'Active'
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '2024-03-24'###DATE change here to D-1
)
"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(f"The count of rows and columns in the extracted data are\t:{df.shape}")


Job ID 354aba9e-8344-4a32-97da-c510bfe43836 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The count of rows and columns in the extracted data are	:(475438, 32)


In [8]:
pd.set_option('display.max_columns', None)
df.sample(10)

,Customer_Id,TSA_Opening_date,TSA_Account_Number,OfblockAmount,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,ARI_Flag,Active_Loan,Total_Loan_Outstanding,Approved_Loans,Pending_Loan_Applications,Hard_Reject_Loans_App,Soft_Reject_Loans_App,Cancelled_Loans_App,Last_Login_Date,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt,ACTION_TYPE
317291,1880134,2023-01-31,60818801340002,0.0,Active,0.00,0.00,0.00,0.0,0,1970-01-01,1970-01-01,N,0,0.00,0,4,0,0,4,2023-06-09,NaT,NaT,NaN,2024-03-24,Batch_1,March,4,0,2023-02-16,NaT,NO ACTION
307198,2269774,2023-10-12,60822697740005,0.0,Active,0.26,0.00,0.00,0.0,0,2024-03-15,2024-03-15,N,1,3463.89,1,0,0,0,0,2024-03-24,2024-03-24,NaT,NaN,2024-03-24,Batch_1,March,0,0,NaT,NaT,NO ACTION
67308,1820259,2022-12-15,60818202590004,0.0,Active,0.00,0.00,0.00,0.0,0,1970-01-01,1970-01-01,N,0,0.00,0,1,0,0,0,2023-01-12,NaT,NaT,NaN,2024-03-24,Batch_1,March,0,0,NaT,NaT,NO ACTION
400785,1278610,2022-03-18,60812786100001,0.0,Active,0.00,0.00,0.00,0.0,0,1970-01-01,1970-01-01,N,0,0.00,0,1,0,0,0,2022-03-18,NaT,NaT,NaN,2024-03-24,Batch_1,March,1,0,2022-08-03,NaT,NO ACTION
152019,2052912,2023-05-20,60820529120002,0.0,Active,336.60,0.00,0.00,0.0,0,2024-02-29,2024-02-29,N,0,0.00,1,0,0,0,0,2023-11-01,2024-03-24,NaT,NaN,2024-03-24,Batch_1,March,0,0,2023-11-20,NaT,NO ACTION
474109,2109664,2023-06-27,60821096640004,0.0,Active,51256.58,0.00,0.00,0.0,0,2024-02-29,2024-02-29,N,0,0.00,0,1,0,0,1,2023-10-09,2024-03-24,NaT,NaN,2024-03-24,Batch_1,March,1,1,2023-07-10,2023-10-10,NO ACTION
366251,1471498,2022-07-17,60814714980002,0.0,Active,0.22,1643.46,3724.53,51000.0,1,2024-02-29,2024-02-29,N,0,0.00,0,0,0,0,0,2024-02-21,2024-03-24,2024-02-29,3724.53,2024-03-24,Batch_1,March,0,0,NaT,NaT,NO ACTION
24500,2384159,2024-01-30,60823841590004,0.0,Active,0.00,0.00,0.00,0.0,0,1970-01-01,1970-01-01,N,0,0.00,0,1,0,1,1,2024-01-30,NaT,NaT,NaN,2024-03-24,Batch_1,March,1,0,2024-01-30,NaT,NO ACTION
407888,1740041,2022-10-28,60817400410002,0.0,Active,206.69,12343.40,0.00,0.0,0,2024-03-16,2024-03-16,N,0,0.00,0,3,0,0,3,2024-03-21,2024-03-24,NaT,NaN,2024-03-24,Batch_1,March,3,1,2023-05-25,2023-08-31,NO ACTION
164686,1760880,2022-11-07,60817608800003,0.0,Active,0.00,0.00,0.00,0.0,0,1970-01-01,1970-01-01,N,0,0.00,0,1,0,0,0,2023-10-25,NaT,NaT,NaN,2024-03-24,Batch_1,March,1,2,2022-12-08,2024-01-24,NO ACTION


In [9]:
result_pivot = df.pivot_table(
    index='ACTION_TYPE',
    columns = 'TSA_Status',
    values=['Customer_Id'],
    aggfunc=pd.Series.nunique
)

# Display the result
# 
result_pivot = result_pivot.reset_index()
result_pivot

,ACTION_TYPE,Customer_Id
TSA_Status,,Active
0,NO ACTION,456778
1,TYPE 1,14154
2,TYPE 2,37
3,TYPE 3,4457


In [10]:
df.to_excel(r"D:\MyStuff\Marvin\TSA Account Closure\20240325_TSA_Account_Closure_1stNotification.xlsx", sheet_name='DATA', index = False)